In [ ]:
import sys, re, json
from os import listdir,path, makedirs
from PIL import Image

with open('../elvconfig.json') as cfg_data:
    cfg = json.load(cfg_data)

####################################################################
inputpath = cfg['inputpath']
outputpath = cfg['outputpath']
startpage = cfg['startpage']
tilecols = cfg['tilecols']
endrow = cfg['endrow']
startrow = cfg['startrow']
per_page = (endrow-startrow+1)*tilecols

In [ ]:
pat = r'.*\((.*\d)\).png'
items = [f for f in listdir(inputpath) if re.match(pat,f)]
items.sort(key=lambda f: int(''.join(list(filter(str.isdigit,f)))))
images = list(map(lambda item: Image.open(path.join(inputpath,item)),items))

if not path.exists(outputpath):
    makedirs(outputpath)

for im in images:
    idx = images.index(im)
    pagenum = int(idx//per_page) + startpage
    filename = '_(' + str(pagenum) + ').png'
    x_offset, y_offset = 0,0
    
    if idx % per_page:  
        # for other 
        co = idx % tilecols
        ro = int((idx // tilecols) % (per_page/tilecols))
        x_offset = im.size[0] * co
        y_offset = im.size[1] * ro
        new_im.paste(im,(x_offset,y_offset))
    else:
        # for first image in the group
        width, height = im.size
        total_width = width * tilecols
        max_height = height * int(per_page/tilecols)
        new_im = Image.new(mode='RGB', size=(total_width, max_height))
        new_im.paste(im,(x_offset,y_offset))

    if (idx % per_page) == (per_page-1):
        new_im.save(path.join(outputpath,filename),'PNG')
        print(idx % per_page)
print("Finished")